<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/eigensolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Variational Quantum Eigensolver (VQE) for Quantum Molecule Simulation**

In [1]:
import sympy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # %matplotlib inline

In [ ]:
!pip install cirq --quiet
import cirq
from cirq.contrib.svg import SVGCircuit
print(cirq.__version__)

### <font color="blue">**VQE to find ground state of hydrogen molecule $H_2$**

Tutorial from [Pennylane: A brief overview of VQE](https://pennylane.ai/qml/demos/tutorial_vqe.html)

See also alternative tutorial: https://joshuagoings.com/2020/08/20/VQE/

###### <font color="blue"> **Preparation: Schrödinger equation can be used to describe the hydrogen atom (not molecule yet!)**

Using a three-dimensional approach, the following form of the Schrödinger equation can be used to describe the hydrogen atom (Three dimensional Schrödinger equation as applied to the H atom):

> $E \psi=-\frac{\hbar^{2}}{2 \mu} \nabla^{2} \psi-\frac{e^{2}}{4 \pi \epsilon_{0} r} \psi$

Compare with time-dependent Schroedinger equation:

> $ E \psi(\vec{r}) = 
\left[-\frac{\hbar^{2}}{2 m} \nabla^{2}+V(\vec{r})\right] \psi(\vec{r}) = -\frac{\hbar^{2}}{2 m} \nabla^{2}\psi(\vec{r}) +V(\vec{r}) \psi(\vec{r})
$

where μ is the reduced mass of the electron-proton pair. **The wavefunction itself is expressed in spherical polar coordinates**:

> $\psi(r, \theta, \phi)=R(r) Y_{l}^{m}(\theta, \phi)=R(r) \Theta(\theta) \Phi(\phi)$

See also: [Schrodinger equation for Hydrogen atom](https://www.youtube.com/watch?v=Gu8VS4BN98o)

where R are radial functions and theta (θ) and phi (ϕ) are spherical harmonic terms. The generalized family of solutions is (family = Also known as a group, a column of elements in the periodic table that share similar reactivity due to their similar valence shell electron configuration):

> $\psi_{n, l, m}(r, \theta, \phi)=\sqrt{\left(\frac{2}{n a_{0}}\right)^{3} \frac{(n-l-1) !}{2 n[(n+l) !]^{3}}} e^{-r / n a_{0}}\left(\frac{2 r}{n a_{0}}\right)^{l} L_{n-l-1}^{2 l+1}\left(\frac{2 r}{n a_{0}}\right) \cdot Y_{l}^{m}(\theta, \phi)$

where $a_0$ is the Bohr radius, L are the generalized Laguerre polynomials, and n, l, and m are the principal, azimuthal, and magnetic quantum numbers, respectively.

###### <font color="blue"> **1. Build the molecular (electronic) Hamiltonian using a minimal basis set approximation.**

*The first step is to specify the molecule we want to simulate. This is done by providing a list with the symbols of the constituent atoms and a one-dimensional array with the corresponding nuclear coordinates in [(Hartree) atomic units](https://en.wikipedia.org/wiki/Hartree_atomic_units):*

```python
from pennylane import numpy as np 
symbols = ["H", "H"] 
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
```

*(not sure if that's helpful here: https://en.m.wikipedia.org/wiki/Born–Oppenheimer_approximation)*

*The molecular structure can also be imported from an external file using the read_structure() function. Now, we can build the electronic Hamiltonian of the hydrogen molecule using the molecular_hamiltonian() function:*

```python
import pennylane as qml

H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)
```

*The outputs of the function are the **Hamiltonian, represented as a linear combination of Pauli operators**, and the number of qubits required for the quantum simulations:*

```python
Number of qubits =  4
The Hamiltonian is    (-0.2427450172749822) [Z2]
+ (-0.2427450172749822) [Z3]
+ (-0.04207254303152995) [I0]
+ (0.17771358191549907) [Z0]
+ (0.17771358191549919) [Z1]
+ (0.12293330460167415) [Z0 Z2]
+ (0.12293330460167415) [Z1 Z3]
+ (0.16768338881432715) [Z0 Z3]
+ (0.16768338881432715) [Z1 Z2]
+ (0.17059759240560826) [Z0 Z1]
+ (0.17627661476093917) [Z2 Z3]
+ (-0.04475008421265302) [Y0 Y1 X2 X3]
+ (-0.04475008421265302) [X0 X1 Y2 Y3]
+ (0.04475008421265302) [Y0 X1 X2 Y3]
+ (0.04475008421265302) [X0 Y1 Y2 X3]
```

*For this example, we use a [minimal basis set](https://de.m.wikipedia.org/wiki/STO-NG-Basissätze) to represent the [molecular orbitals](https://en.m.wikipedia.org/wiki/Molecular_orbital). In this approximation, we have four spin orbitals, which defines the number of qubits. Furthermore, we use the Jordan-Wigner transformation to perform the fermionic-to-qubit mapping of the Hamiltonian. For a more comprehensive discussion on how to build the Hamiltonian of more complicated molecules, see the tutorial [Building molecular Hamiltonians](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html).*

###### <font color="blue"> **2. Design the quantum circuit preparing the trial state of the molecule**

*Define the device, in this case PennyLane’s standard qubit simulator:*

```python
dev = qml.device("default.qubit", wires=qubits)
```

*Next, define the quantum circuit that prepares the trial state of the molecule. We want to prepare states of the form:*

> <font color="blue">$|\Psi(\theta)\rangle=\cos (\frac{\theta}{2})|1100\rangle-\sin (\frac{\theta}{2})|0011\rangle$

*where $\theta$ is the variational parameter to be optimized in order to find the best approximation to the true ground state. In the [Jordan-Wigner encoding (transformation)](https://de.m.wikipedia.org/wiki/Jordan-Wigner-Transformation)*

  * *the first term |1100⟩ represents the [Hartree-Fock (HF) state](https://en.wikipedia.org/wiki/Hartree–Fock_method) where the two electrons in the molecule occupy the lowest-energy orbitals.*
  
  * *the second term |0011⟩ encodes a double excitation of the HF state where the two particles are excited from qubits 0, 1 to 2, 3*

*The quantum circuit to prepare the trial state $|\Psi (\theta) \rangle$ is schematically illustrated in the figure below:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_304.png)

*In this figure, the gate $G^{(2)}$ corresponds to the DoubleExcitation operation, implemented in PennyLane as a Givens rotation, which couples the four-qubit states |1100⟩ and |0011⟩. For more details on how to use the excitation operations to build quantum circuits for quantum chemistry applications see the tutorial Givens rotations for quantum chemistry.*

*Implementing the circuit above using PennyLane is straightforward. First, we use the hf_state() function to generate the vector representing the Hartree-Fock state.*

```python
electrons = 2
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)
```

*The output is then:*

```python
[1 1 0 0]
```

*The hf array is used by the BasisState operation to initialize the qubit register. Then, we just act with the DoubleExcitation operation on the four qubits:*

```python
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
```

###### <font color="blue"> **3. Design the cost function to evaluate the expectation value of the Hamiltonian**

*The next step is to define the cost function to compute the expectation value of the molecular Hamiltonian in the trial state prepared by the circuit. We do this using the expval() function. The decorator syntax allows us to run the cost function as an executable QNode with the gate parameter $\theta$:*

```python
@qml.qnode(dev)
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)
```

###### <font color="blue"> **4. Select a classical optimizer, initialize the circuit parameters, and run the VQE algorithm using a PennyLane simulator.**

*Now we proceed to minimize the cost function to find the ground state of the $H_2$ molecule. To start, we need to define the classical optimizer. PennyLane offers many different built-in optimizers. Here we use a basic gradient-descent optimizer.*

```python
opt = qml.GradientDescentOptimizer(stepsize=0.4)
```

*We carry out the optimization over a maximum of 100 steps aiming to reach a convergence tolerance of $10^{-6}$ for the value of the cost function:*

```python
# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")
```

*The output is then:*

```python
Step = 0,  Energy = -1.12799983 Ha
Step = 2,  Energy = -1.13466246 Ha
Step = 4,  Energy = -1.13590595 Ha
Step = 6,  Energy = -1.13613667 Ha
Step = 8,  Energy = -1.13617944 Ha
Step = 10,  Energy = -1.13618736 Ha
Step = 12,  Energy = -1.13618883 Ha

Final value of the ground-state energy = -1.13618883 Ha

Optimal value of the circuit parameter = 0.2089
```

*Let’s plot the values of the ground state energy of the molecule and the gate parameter $\theta$ as a function of the optimization step:*

```python
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(12)

# Full configuration interaction (FCI) energy computed classically
E_fci = -1.136189454088

# Add energy plot on column 1
ax1 = fig.add_subplot(121)
ax1.plot(range(n + 2), energy, "go", ls="dashed")
ax1.plot(range(n + 2), np.full(n + 2, E_fci), color="red")
ax1.set_xlabel("Optimization step", fontsize=13)
ax1.set_ylabel("Energy (Hartree)", fontsize=13)
ax1.text(0.5, -1.1176, r"$E_\mathrm{HF}$", fontsize=15)
ax1.text(0, -1.1357, r"$E_\mathrm{FCI}$", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add angle plot on column 2
ax2 = fig.add_subplot(122)
ax2.plot(range(n + 2), angle, "go", ls="dashed")
ax2.set_xlabel("Optimization step", fontsize=13)
ax2.set_ylabel("Gate parameter $\\theta$ (rad)", fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.subplots_adjust(wspace=0.3, bottom=0.2)
plt.show()
```

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_305.png)

*In this case, the VQE algorithm converges after thirteen iterations. The optimal value of the circuit parameter $\theta^*$ = 0.208 defines the state:*

> <font color="blue">$\left|\Psi\left(\theta^{*}\right)\right\rangle=0.994|1100\rangle-0.104|0011\rangle$

*which is precisely the ground state of the $H_2$ molecule in a minimal basis set approximation.*

*For comparison, remember in step 2 the trial state was:*

> <font color="blue">$|\Psi(\theta)\rangle=\cos (\frac{\theta}{2})|1100\rangle-\sin (\frac{\theta}{2})|0011\rangle$



**Conclusion**

*In this tutorial, we have implemented the VQE algorithm to find the ground state of the hydrogen molecule. We used a simple circuit to prepare quantum states of the molecule beyond the Hartree-Fock approximation. The ground-state energy was obtained by minimizing a cost function defined as the expectation value of the molecular Hamiltonian in the trial state.*

*The VQE algorithm can be used to simulate other chemical phenomena. In the tutorial tutorial_vqe_bond_dissociation, we use VQE to explore the potential energy surface of molecules to simulate chemical reactions. Another interesting application is to probe the lowest-lying states of molecules in specific sectors of the Hilbert space. For example, see the tutorial [VQE in different spin sectors](https://pennylane.ai/qml/demos/tutorial_vqe_spin_sectors.html). Furthermore, the algorithm presented here can be generalized to find the equilibrium geometry of a molecule as it is demonstrated in the tutorial [Optimization of molecular geometries](https://pennylane.ai/qml/demos/tutorial_mol_geo_opt.html).*



### <font color="blue">**Introduction into Variational Methods**

> Variational Methods: exist for approaching the ground state energy of a system

###### **Links**

https://qiskit.org/textbook/ch-applications/vqe-molecules.html

https://medium.com/qiskit/building-a-quantum-variational-classifier-using-real-world-data-809c59eb17c2

https://pennylane.ai/qml/demos/tutorial_vqe.html

https://arxiv.org/abs/2012.09265

https://pennylane.ai/qml/demos/tutorial_quantum_natural_gradient.html

https://pennylane.ai/qml/glossary/variational_circuit.html#glossary-variational-circuit

https://pennylane.ai/qml/glossary/parameter_shift.html

https://de.wikipedia.org/wiki/Downhill-Simplex-Verfahren

###### **Variational Principle**

https://en.m.wikipedia.org/wiki/Many-body_problem

In science and especially in mathematical studies, a [variational principle](https://en.m.wikipedia.org/wiki/Variational_principle) is one that enables a problem to be solved using calculus of variations, which concerns finding functions that optimize the values of quantities that depend on those functions.

In [quantum mechanics, the variational method](https://en.m.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) is one way of finding approximations to the lowest energy eigenstate or ground state, and some excited states. This allows calculating approximate wavefunctions such as molecular orbitals. The basis for this method is the variational principle.

  * The method consists of choosing a "trial wavefunction" depending on one or more parameters, and finding the values of these parameters for which the expectation value of the energy is the lowest possible. 
  
  * The wavefunction obtained by fixing the parameters to such values is then an approximation to the ground state wavefunction, and the expectation value of the energy in that state is an upper bound to the ground state energy. **The [Hartree–Fock method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method), [Density matrix renormalization group](https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group), and [Ritz method](https://en.m.wikipedia.org/wiki/Ritz_method) apply the variational method**.

  * This principle relies on the fact that a Hermitian is equal to its own conjugate transpose (ie 𝐻=𝐻†), and since any eigenvalue of H must be real, all eigenvalues of 𝐻 have the property of 𝜆𝑖=𝜆∗.

###### **Mean-field theory**

* In physics and probability theory, [mean-field theory](https://en.m.wikipedia.org/wiki/Mean-field_theory) (aka MFT or rarely self-consistent field theory) studies the behavior of high-dimensional random (stochastic) models by studying a simpler model that approximates the original by averaging over degrees of freedom (the number of values in the final calculation of a statistic that are free to vary). Such models consider many individual components that interact with each other. 

* In MFT, the effect of all the other individuals on any given individual is approximated by a single averaged effect, **thus reducing a [many-body problem](https://en.m.wikipedia.org/wiki/Many-body_problem) to a [one-body problem (two body)](https://en.m.wikipedia.org/wiki/Two-body_problem)**.

###### **Ritz Method**

* The [Ritz method](https://en.wikipedia.org/wiki/Ritz_method) is a direct method to find an approximate solution for [boundary value problems](https://en.wikipedia.org/wiki/Boundary_value_problem). The method is named after Walther Ritz, although also commonly called the [Rayleigh–Ritz method](https://en.wikipedia.org/wiki/Rayleigh–Ritz_method) and the [Ritz-Galerkin method](https://en.wikipedia.org/wiki/Galerkin_method).

* In quantum mechanics, a system of particles can be described in terms of an "energy functional" or Hamiltonian, which will measure the energy of any proposed configuration of said particles. 
  
  * It turns out that certain privileged configurations are more likely than other configurations, and this has to do with the eigenanalysis ("analysis of characteristics") of this Hamiltonian system.
  
  * Because it is often impossible to analyze all of the infinite configurations of particles to find the one with the least amount of energy, it becomes essential to be able to approximate this Hamiltonian in some way for the purpose of numerical computations.

* The Ritz method can be used to achieve this goal. In the language of mathematics, it is exactly the [finite element method](https://en.wikipedia.org/wiki/Finite_element_method) used to compute the eigenvectors and eigenvalues of a Hamiltonian system.


https://www.youtube.com/watch?v=w4ijyrdLbNA

https://en.m.wikipedia.org/wiki/Ritz_method

https://en.m.wikipedia.org/wiki/Rayleigh%E2%80%93Ritz_method

###### **Hartree–Fock Method**

* In computational physics and chemistry, the [Hartree–Fock (HF) method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) is a method of approximation for the determination of the wave function and the energy of a [quantum many-body system](https://en.m.wikipedia.org/wiki/Many-body_problem) <u>**in a stationary state**</u>.

* The Hartree–Fock method often assumes that the exact N-body wave function of the system can be approximated 

  * by a single [Slater determinant](https://en.m.wikipedia.org/wiki/Slater_determinant) (in the case where the particles are fermions) or 
  
  * by a single [permanent](https://en.m.wikipedia.org/wiki/Permanent_(mathematics)) (in the case of bosons) of N [spin-orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital). 

* By invoking the [variational method](https://en.m.wikipedia.org/wiki/Calculus_of_variations), one can derive a set of N-coupled equations for the N spin orbitals. A solution of these equations yields the Hartree–Fock wave function and energy of the system.





https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method

https://en.m.wikipedia.org/wiki/Slater_determinant

https://en.m.wikipedia.org/wiki/Permanent_(mathematics)

https://quantumai.google/cirq/experiments/hfvqe

###### **Density matrix renormalization group**

https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group

### <font color="blue">**Mathematics of Variational Eigensolvers**

> Model different molecules and how they interact together

* **Objective**: create new, more effective medicines along with researching how medications might react with diseases such as cancer.

* **Challenge**: effectively make the calculations needed for the chemical reactions that occur. 

  * One such calculation is **calculating the ground state energy of different small molecules**.

  * Running this calculation on a classical computer can take exponential time to get an exact numerical answer.


* *Exkurs: The [ground state energy](https://en.wikipedia.org/wiki/Excited_state) is the lowest possible energy state that a molecule can be in. This is important because it gives more information about the electron configuration of the molecule.*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Energy_levels.svg/306px-Energy_levels.svg.png)


**From chemistry and physics we know that the Schrodinger equation tells us that if $H$ is the Hamiltonian of a system, $H|ψ⟩ = E_g|ψ⟩$ where the minimum eigenvalue $E_g$ that solves this equation corresponds to the ground state energy of the molecule**.

* To find this energy, we want to **find the exact minimum eigenvalue** that corresponds to that Hermitian matrix. 

* However that **would require a perfectly working quantum computer using the QPE (Quantum Phase Estimation Algorithm)**, which is not possible as of right now.

* Rather, we can use the [variational principle](https://en.wikipedia.org/wiki/Variational_principle) to find a close, if not exact solution.






This principle relies on the fact that a Hermitian is equal to its own conjugate transpose (ie 𝐻=𝐻†), 

* and since any eigenvalue of H must be real, all eigenvalues of 𝐻 have the property of 𝜆𝑖=𝜆∗. 

* We can rewrite the Hermitian as follows, where 𝜆 is the eigenvalue associated with the eigenvector |𝜓_i⟩

> $H=\sum_{i=1}^{N} \lambda_{i}\left|\psi_{i}\right\rangle\left\langle\psi_{i}\right|$

* which we can then use to define the expectation of this Hermitian as

> $\langle H\rangle_{\psi} \equiv\langle\psi|H| \psi\rangle$

* And finally we can substitute in for $H$ and get the following

> $\begin{aligned}\langle H\rangle_{\psi}=\langle\psi|H| \psi\rangle &=\left\langle\psi\left|\left(\sum_{i=1}^{N} \lambda_{i}\left|\psi_{i}\right\rangle\left\langle\psi_{i}\right|\right)\right| \psi\right\rangle \\ &=\sum_{i=1}^{N} \lambda_{i}\left\langle\psi \mid \psi_{i}\right\rangle\left\langle\psi_{i} \mid \psi\right\rangle \\ &=\sum_{i=1}^{N} \lambda_{i}\left|\left\langle\psi_{i} \mid \psi\right\rangle\right|^{2} \end{aligned}$

* which demonstrates that **any state can be described using a sum of the eigenvalues of the Hermitian as weights**. 

* One final property of the Hermitian is what truly allows the variational principle to work is that the eigenvalues of it have to be real, non-negative numbers, such that

> $\lambda_{\min } \leq\langle H\rangle_{\psi}=\langle\psi|H| \psi\rangle$

* And this equation is what gives power to the Variational Quantum Eigensolver. 

> **Since the minimum eigenvalue can never be smaller than the expectation value that we are looking for, then by minimizing our expectation, we will get really close to the minimum eigenvalue**.

https://medium.com/mit-6-s089-intro-to-quantum-computing/quantum-molecule-simulation-using-vqe-d34b9c651e3d

### <font color="blue">**Variational Quantum Algorithms**

###### <font color="blue">**Basic Approach**

> quantum/classical hybrid algorithm that can be used to find eigenvalues of a (often large) matrix H

* In this hybrid algorithm a quantum subroutine is run inside of a classical optimization loop.

The quantum subroutine has two fundamental steps:

1. Prepare the quantum state $|\Psi(\operatorname{vec}(\theta))\rangle$, often called the ansatz.
2. Measure the expectation value $\langle\Psi(\operatorname{vec}(\theta))|H| \Psi(\operatorname{vec}(\theta))\rangle$.

* The [variational principle](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) ensures that this expectation value is always greater than the smallest eigenvalue of H.

This bound allows us to use classical computation to run an optimization loop to find this eigenvalue:

* Use a classical non-linear optimizer to minimize the expectation value by varying ansatz parameters vec(θ).
* Iterate until convergence.

**Lecture: [Quantum Variational Eigensolver](https://www.youtube.com/watch?v=UBkb54_MOLo)**

* You could actually use Quantum Phase Estimation (it is exact!), but there are two problems:
  * you need an accurate approximate wavefunction as an input
  * The number of gates represent the unitary operator $U=e^{i t \hat{H}} \approx \prod_{j} U_{j}^{(g)}$
    * Quantum computers cannot implement exponents of hamiltonian as one gate for interesting problems, 
    * you need to break this unitary transformation into a product of elementary gates, 
    * a sequence of those gates becomes too long to implement on any currently available computer

* In order to address two of these problems the variational quantum eigensolver was suggested

  * $\mathrm{E}_{\mathrm{e}}=\min _{\theta}<0\left|\mathrm{U}(\theta)^{\prime} \mathrm{HU}(\theta)\right| 0>$

  * trying to optimize the unitary transformation directly so that the energy would be minimized, 

  * instead of (like in QPE) setting up a particular unitary transformation and then breaking it donw into gates

* The VQE is a hybrid eigensolver, where the minimization of energy is split into two parts:

  * The quantum computer sets up the quantum wavefunction: it starts with qubits in some vaccum state and then rotates the qubits, then entangles them, essentially obtains the sum (trial) wave function, and measures the Hamiltonian expectation value on this wavefunction

  * The result of the expectation value is passed to the classical computer which tries to come up with new angles of the unitary rotations so that on the next iteration we could lower the expectation value even more: $\min _{\theta}<\Phi_{q}(\theta)\left|H_{q}\right| \Phi_{q}(\theta)>$

  * The classical computer gives to the quantum computer a new trial wave function that the quantum computer needs to implement and obtain the expectation value

  * This cyle repeats and eventually the classical computer will converge to some value that will be the lowest

* Challenges:

  * Unitary transformation $U (\theta)$ is not easy to find because it lives in an exponentially larege space of all possible unitary trransformations

  * Entire $H_q$ measurement cannot be done at once, you partition the Hamiltonian in pieces which you need to measure separately

  * The Hilbert space of the qubit Hamiltonian $H_q$ is essentially equivalent to the entire Fock space of the original problem, that contains various electronic states (vraious numbers of electrons, spins etc), and looking for one particular electronic state would be challenging

**Video: [How to design a quantum variational eigensolver?](https://www.youtube.com/watch?v=bwmLfxelwUA)**

Generic Structure:
* quantum computer efficiently computes cost function and sends cost to classical computer
* Classical computer adjusts the parameters of the Ansatz to reduce cost and sends new parameters to quantum computer

How does it work? Find the ground state of some Hamiltonian (for example solving for the ground states of some molecule) 

* First step: express the Hamiltonian as a linear combination of Pauli operators
* $\begin{aligned} \mathcal{H} &=\sum_{i \alpha} h_{\alpha}^{i} \sigma_{\alpha}^{i}+\sum_{i j \alpha \beta} h_{\alpha \beta}^{i j} \sigma_{\alpha}^{i} \sigma_{\beta}^{j}+\ldots \\\langle\mathcal{H}\rangle &=\sum_{i \alpha} h_{\alpha}^{i}\left\langle\sigma_{\alpha}^{i}\right\rangle+\sum_{i j \alpha \beta} h_{\alpha \beta}^{i j}\left\langle\sigma_{\alpha}^{i} \sigma_{\beta}^{j}\right\rangle+\ldots \end{aligned}$
* We assume that the number of terms in that expansion is small, growing only polynomial in the system size
* That’s necessary because what you‘re going to do in order to evaluate the expectation value for the Hamiltonian is you gonna do that term by term, where you evaluate the expectation value of each individual Pauli operator on the quantum computer, then you classically sum them up with the appropriate weight
* You first prepare a guess for that ground state, and then you measure the invidual terms of the Hamiltonian and then you sum them up classically and then you have a classical feedback loop where you adjust the parameters of the gate sequence and you feed the new parameters to the quantum computer which then repeats the loop over and over again
* The end result is that you‘ve prepared a state that even closer to the ground state than your original guess
* There are many applications for this, for example for electronic structure but also for nuclear structure
    * For example in one paper they solve for the ground state of the deuterum Hamiltonian, starting with the fermionic hamiltonian, 
    * $H_{N}=\sum_{n, n^{\prime}=0}^{N-1}\left\langle n^{\prime}|(T+V)| n\right\rangle a_{n^{\prime}}^{\dagger} a_{n}$
    * they then transform it into Pauli operators
    * $a_{n}^{\dagger} \rightarrow \frac{1}{2}\left[\prod_{j=0}^{n-1}-Z_{j}\right]\left(X_{n}-i Y_{n}\right)$ (Jordan Wigner transformation)
    * And then they have a simple ansatz for the ground state, and then they train the parameters of this Ansatz in order to solve for the ground state energy
    * And they find good agreement bvetween the ground state energy from the quantum computer versus the one they get from the exact classical diagonalization

###### <font color="blue">**Variational Quantum Algorithms**

Arxiv Paper: [Variational Quantum Algorithms](https://arxiv.org/abs/2012.09265)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_187.png)

Source: [Variational Quantum Algorithms](https://www.youtube.com/watch?v=YtepXvx5zdI)

###### <font color="blue">**Ansatz**

how noise can influence which ansatz family is best for a quantum chemistry problem: https://medium.com/qiskit/the-effect-of-noise-on-the-performance-of-variational-algorithms-for-quantum-chemistry-9cac4526abc1

> In the context of variational circuits, an ansatz usually describes a subroutine consisting of a sequence of gates applied to specific wires. Similar to the architecture of a neural network, this only defines the base structure, while the types of gates and/or their free parameters can be optimized by the variational procedure. 

**One can distinguish three different base structures, namely**

* a layered gate ansatz, 

* an alternating operator ansatz, 

* a tensor network ansatz.

**The ansatz can come from:**

- Some basis in physics, chemistry, or quantum information theory (e.g., VQE)
- The structure of the problem (e.g., QAOA)
- Intuition borrowed from machine learning
- No place at all (use your imagination!)



> The choice of ansatz affects the model/function that can be learned (more layers often better)

https://pennylane.ai/qml/glossary/circuit_ansatz.html

https://medium.com/arnaldo-gunzi-quantum/what-is-ansatz-31e682b0518b

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_188.png)

https://en.m.wikipedia.org/wiki/Ansatz

https://de.m.wikipedia.org/wiki/Ansatz_(Mathematik)

###### <font color="blue">**Embedd Classical Data into a Quantum Variational Circuit**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_189.png)

> **The easiest way for a parameter to enter a circuit is through a rotation of a single qubit, in proportion to the value of a single datapoint, so a single scalar value:**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_190.png)

> **You can also use a sequence of rotations to embedd data (reuploding).** And maybe there is free parameters in between as well. Can make a more complex function available than if you upload only once in a single rotation.

> **Learnable embeddings**: The other idea is to actually have a trainable embedding layer. Not to worry about training the unitary of the circuit, but worry about training the embedding and then use standard quantum information metrics to classify the data.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_191.png)

###### <font color="blue">**Optimizer**

> When we're using variational circuits, if you can compute the gradients the parameter shift rule, then that opens up every possible flavor of gradient descent. Like standard gradient descent, but in deep learning also all sorts of gradient descent optimizers. Most common: momentum and Adam.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_192.png)

<font color="blue">**But there are also quantum-aware optimizers!**

**Rotosolve**: don't use gradient descent, but sinusoid structure. You eventually end up in a global minima via a sequence of individual jumps to local minima.

[**Quantum Natural Optimizer**](https://pennylane.ai/qml/demos/tutorial_quantum_natural_gradient.html): Internal geometry of quantum circuits and quantum physical system is not Euclidean like the world around us, like this rectilinear structure. It's more sinusoidal structure. We should take this into account. 

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_193.png)

<font color="blue">**Barren Plateau**

* There are parts of the optimization landscape where the gradient is zero, and around it also zero, like in classical deep learning. everything flat in every direction where you go.

* Barren Plateau can come from number of different effects: choice from circuit ansatz, choice of parametrization (parameter values), or from cost function

* several proposals to overcome this

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_194.png)

###### <font color="blue">**Step by Step Algorithm**

This algorithm is a **hybrid machine learning quantum computer algorithm**, where we run part of it on a quantum computer, and then finish off the minimization classically (since a solely quantum algorithm provides no speedups).

**Step 1: We encode the molecular Hamiltonian onto the qubits**

* the variational form typically uses the U3 gate (QASM?), where we can describe it with the following matrix:

> $U 3(\theta, \phi, \lambda)=\left(\begin{array}{cc}\cos \left(\frac{\theta}{2}\right) & -e^{i \lambda} \sin \left(\frac{\theta}{2}\right) \\ e^{i \phi} \sin \left(\frac{\theta}{2}\right) & e^{i \lambda+i \phi} \cos \left(\frac{\theta}{2}\right)\end{array}\right)$

* We can then construct a quantum circuit that encodes the information that we want, but it is important that this circuit (and these gates as a whole) allow for any universal state to be entered and returned, which can result in more complex circuits.

* The end result of this encoding allows us to make calculations with the qubits, but now the qubits are mimicking the electron orbital interactions and so it almost as if we are making the computations we need with the molecule itself.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_181.png)

**Step 2: Create the ansatz**

* An ansatz, in simplest terms, is a really good mathematical guess that can cover many possible states. 

* For our purposes, the thing that we will be guessing will be what the electron configuration is for the specific molecule at a given interatomic distance between each atom and thus the ansatz can be described as our trial wavefunction. 

* We also require that the ansatz be shallow due to the nature of programs on quantum computers needing to run quickly or else they’ll be more prone to errors.

> In the context of variational circuits, an ansatz usually describes a subroutine consisting of a sequence of gates applied to specific wires. Similar to the architecture of a neural network, this only defines the base structure, while the types of gates and/or their free parameters can be optimized by the variational procedure. **One can distinguish three different base structures, namely a layered gate ansatz, an alternating operator ansatz, and a tensor network ansatz.**

https://pennylane.ai/qml/glossary/circuit_ansatz.html

https://medium.com/arnaldo-gunzi-quantum/what-is-ansatz-31e682b0518b


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_183.png)

**Step 3: Calculate our trial state energy**

* use the information that we have to calculate the ground state energy level of our ansatz. 

* The calculation uses the information given by the Hamiltonian for a given inter-atomic distance, and the energy for that particular electron configuration.

**Step 4: The Classical Side**

> Task a: Measure and optimize our parameter

* The first step here is to measure our qubits such that we get the information that we wanted from the quantum steps. 

* We then use machine learning algorithms to optimize our parameters such that our cost function, which represents the ground state energy, is minimized.

* In terms of the optimization, the [Simultaneous Perturbation Stochastic Approximation optimizer (SPSA)](https://en.m.wikipedia.org/wiki/Simultaneous_perturbation_stochastic_approximation) is typically used due to how noisy the objective function can be. One thing that we had considered using during our experimentation with the VQE algorithm was gradient descent. 

* We noticed that our original code was just an iteration over the possible values that the inter-atomic distance could take and we thought that gradient descent might be able to speed this process up by just choosing a good start value. 

* However, what we found was that this strategy was prone to getting stuck in local optimum and through the reading of the Qiskit textbook and other research papers, we found that it is in fact a costly optimization in terms of the number of circuit evaluations performed.

* So then we questioned how does SPSA work and why does it not have the same pitfalls as gradient descent? They both share the similarity that both optimizes approximate the gradient of our cost function, however, SPSA “does so by concurrently perturbing all of the parameters in a random fashion, in contrast to gradient descent where each parameter is perturbed independently” as discussed in the Qiskit textbook tutorial.



> Task b: Iterate using the resulting parameters

* The second step of the classical side of this algorithm is that we then take the optimized parameters that we got and feed them into the quantum part again, giving us a new set of parameters that should improve the ansatz. 

* This step is iterative because we continually run through out optimizer until we are able to converge onto the lowest energy state for that bond length. 

* We then continue to iterate over the different inter-atomic distances until we find the bond length associated with the molecule’s lowest energy state.


*Results of VQE on RbH where Hartree-Fock is the expected and VQE is what we got from running the algorithm:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_184.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_185.png)

Source: [Variational Quantum Algorithms](https://www.youtube.com/watch?v=YtepXvx5zdI)

###### <font color="blue">**Cirq**

https://quantumai.google/cirq/tutorials/variational_algorithm

In [ ]:
import cirq

# define the length and width of the grid.
length = 3

# define qubits on the grid.
qubits = cirq.GridQubit.square(length)

print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


Now that we have some qubits, let us construct a cirq.Circuit on these qubits. For example, suppose we want to apply the Hadamard gate cirq.H to every qubit whose row index plus column index is even, and an cirq.X gate to every qubit whose row index plus column index is odd. To do this, we write:

In [ ]:
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0)
circuit.append(cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1)

print(circuit)

(0, 0): ───H───

(0, 1): ───X───

(0, 2): ───H───

(1, 0): ───X───

(1, 1): ───H───

(1, 2): ───X───

(2, 0): ───H───

(2, 1): ───X───

(2, 2): ───H───


In [ ]:
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""

    # Define the gate once and then re-use it for each Operation.
    rot = cirq.XPowGate(exponent=half_turns)

    # Create an X rotation Operation for each qubit in the grid.
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

# Create the circuit using the rot_x_layer generator
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


### <font color="blue">**QAOA (Quantum Approximate Optimization Algorithm)**

https://qiskit.org/textbook/ch-applications/qaoa.html

https://en.wikipedia.org/wiki/Quantum_optimization_algorithms

* the cost function is something that is encoding an optimization problem. you might have an (discrete) optimization problem with many clauses that have to be satisfied and you have to encode this into an Ising type model, a spin chain type model, which then can be converted into observables which can be measured on a quantum circuit

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_186.png)

Source: [Variational Quantum Algorithms](https://www.youtube.com/watch?v=YtepXvx5zdI)